# 3.12.28 Pandas Data Wrangling

### Missing Data

Using the `GA Paid Search Traffic.csv` file from the `data` folder, let's load the "third block of data" from the file, that is, the Users' time series data, using the `pd.read_csv()` function.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!cat data/GA\ Paid\ Search\ Traffic.csv

In [ ]:
ts = pd.read_csv("data/GA Paid Search Traffic.csv", skiprows=18, nrows=31)
print(ts.shape)
ts.head()

There are no missing data in this time series, so let's **introduce NAs manually**. To do that, we will assign the `np.nan` object to some of the data points in the `Users` column. To make things more interesting, let's introduce a new method, `.sample()`, which **takes a sample from your DataFrame** (you can decide the size of the sample via the `n` parameter). We can use this sample to randomly select 5 rows of the DataFrame, we will then select the values in the `Users` column and replace them with the `np.nan` object.

In [ ]:
ts.sample(n=5, random_state=42).index

In [ ]:
ts.iloc[ts.sample(n=5, random_state=42).index, 1] = np.nan

First of all, let's **look at these missing values** we just created; in order to filter a DataFrame and show just the missing values, you can use the `.isnull()` method on the DataFrame / Series itself. 

In [ ]:
ts[ts.Users.isnull()]

When you're faced with **missing values** there are several paths you can choose to follow: 

- you can **drop the observations** containing missing values; this is probably the worst option, since you may be missing information, espectially if that row has values in other variables of the DataFrame
- you can **fill them with zeros**; this solution may be useful and realistic in some situations (for instance if the value was missing because there were no sales on that day) and detrimental in others (if there was a technical issue with the recording of that data point, for example).
- you can **fill them with the previous or with the next data point** (or with the mean of previous and next) if data is sequential in nature (like in our example); note that you can't apply this method if, for example, you're looking at city population and each row/observation is a different city
- you can **fill them with the average** of that column / variable
- you can do **some other kind of interpolation, prediction or filtering** via several methods such as [spline interpolation](https://en.wikipedia.org/wiki/Spline_interpolation), [moving average](https://en.wikipedia.org/wiki/Moving_average) methods and, in more complex scenarios, [linear regression](https://en.wikipedia.org/wiki/Linear_regression). 

Let's see how we can fill these missing values using the `.fillna()` method: 

In [ ]:
# fill with zeros
ts_fill = ts.copy()
ts_fill.fillna(0).head(10)

In [ ]:
# fill with next valid observation 
ts_fill.fillna(method='bfill').head(10)

In [ ]:
# fill with last valid observation
ts_fill.fillna(method='ffill').head(10)

In [ ]:
# fill with average of column
ts_fill.fillna(ts_fill.Users.mean()).head(10)

Using the `.interpolate()` method, you can also fill all the missing values using a linear interpolation. Check out the [method parameter](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate) for a list of all the options available. 

In [ ]:
# fill with interpolation --> see the method parameter for more options!
ts_fill.interpolate()

### Joining Data

You can connect rows in DataFrames based on one or more keys via the `pd.merge()` [function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html). These kind of operations are analogous to **join operations** performed on a relational database using the SQL language. 

We will be using the European Soccer Database, a collection of CSV files containing matches data and metadata about the teams and leagues they play in: 

- match.csv
- team.csv
- leagues.csv

Let's start by loading and looking into the first two datasets: 

In [ ]:
match_raw = pd.read_csv("data/European Soccer Database/match.csv")
match_raw.head()

In [ ]:
team = pd.read_csv("data/European Soccer Database/team.csv")
team.head()

By looking at the data, we can see that there are the following **entity relationships** that link each table to one another: 

<img src="img/European Soccer DB - ERD.png" width="600">

For example, let's start by [merging](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) the `match` with the `team` dataset via the key-pairs `home_team_api_id <-> team_api_id` and then on `away_team_api_id <-> team_api_id`: 

In [ ]:
# merge on home team
match = match_raw.merge(team[['team_api_id','team_long_name']], how='left', left_on='home_team_api_id', right_on='team_api_id')
match.head()

In [ ]:
# merge on away team
match = match.merge(team[['team_api_id','team_long_name']], how='left', left_on='away_team_api_id', right_on='team_api_id')
match.head()

The whole thing looks a bit messy, let's clean it up a bit by keeping just the relevant columns: 

In [ ]:
match.columns

I **reorder some of the columns** using the double-bracket `[[ ]]` operator; note that if I omit some of the column names, they will be automatically dropped from the resulting DataFrame.

In [ ]:
# reorder columns in the DataFrame (omitting a column name will implicitly drop that column)
match = match[['match_api_id', 'league_id', 'season', 'stage', 'date', 
               'home_team_api_id', 'away_team_api_id', 'team_long_name_x', 'team_long_name_y', 
               'home_team_goal', 'away_team_goal']]
match.head()

Then I **rename some of the columns** in order to make them easier to read and interpret. Check out the [documentation page](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) of the `.rename()` method for more information. 

In [ ]:
# rename columns in the DataFrame
match = match.rename(columns={"team_long_name_x": "home_team_name", "team_long_name_y": "away_team_name"})
match.head()

### Grouping data

After all this work we have a nicely shaped dataset, so it would be interesting to **investigate it further by grouping and aggregating its variables**. To do this, we can use the `.groupby()` [method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to combine all the elements of the same category and a function like `.size()`, `.sum()` or `.mean()` to aggregate the results over a specific metric. 

Let's say we're interested in finding out the sum and the average home goals scored in the whole dataset; we can use the `.agg()` [method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html): 

In [ ]:
match['home_team_goal'].agg(['min', 'mean', 'sum', 'max', 'count'])

If we want to aggregate these two metrics over each `season` group, we can combine the **group by** with the aggregation method we just introduced: 

In [ ]:
match.groupby('season')['home_team_goal'].agg(['min', 'mean', 'sum', 'max', 'count'])

Check out [this webpage](https://pbpython.com/groupby-agg.html) for more examples on the use of groupby() and agg(). 

If we're interested in the number of matches played in each season (remember that in this dataset, each row represents a different match), we can use the `.size()` method, which **returns the total number of row count for each group** *(note that you don't need to specify a column, you can use it over the whole DataFrame)*: 

In [ ]:
# size is the equivalent of count
match.groupby('season').size()

If, instead, we wanted to find out the **number of matches played in each month** of every `season`, we would need to create a new `month` variable (where we [extract the month from the date](https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas) column) and include it in the group by statement: 

In [ ]:
# extract the month from the date column and save it as a new variable
match['month'] = pd.DatetimeIndex(match['date']).month
match.head()

In [ ]:
df1 = match.groupby(['season', 'month']).size()
df1

Great, this is what we wanted! However, you may have noticed that **it returned a multi-index DataFrame**; I personally prefer working with single-index DataFrames, so if you include the `as_index=False` parameter in the `.groupby()` method, you can flatten the multi-index to a single-index DataFrame: 

In [ ]:
df2 = match.groupby(['season', 'month'], as_index=False).size()
df2.head(11)

Now, let's say we wanted to **find out the total number of goals** (home + away) per team in the most recent available season; we'd need to split the problem into separate subproblems: 

1. first we filter the data to show just the most recent season
2. then we group all the home teams and calculate the total home goals
3. then we group all the away teams and calculate the total away goals
4. finally, we join the two DataFrames and calculate the total number of goals

In [ ]:
# find out which season is the most recent one
match.season.unique()

In [ ]:
match15_16 = match.loc[match['season']=='2015/2016'].copy()

In [ ]:
# create a DataFrame that contains all the home team goals
home = match15_16.groupby(['home_team_name'], as_index=False)['home_team_goal'].sum()
home.head()

In [ ]:
# create a DataFrame that contains all the away team goals
away = match15_16.groupby(['away_team_name'], as_index=False)['away_team_goal'].sum()
away.head()

In [ ]:
# create a DataFrame that contains all the home + away team goals
all_goals = home.merge(away, how='inner', left_on='home_team_name', right_on='away_team_name')
all_goals.head()

In [ ]:
# create a new variable containing the total number of goals (home + away)
all_goals['total_goal'] = all_goals['home_team_goal'] + all_goals['away_team_goal']
all_goals.head()

In [ ]:
# rename variable to team_name
all_goals = all_goals.rename(columns={"home_team_name":"team_name"})

In [ ]:
# drop away_team_name variable
all_goals.drop('away_team_name', axis=1, inplace=True)
all_goals.sort_values('total_goal', ascending=False).head()

### Exercise

Perform the following tasks and /or answer to the following questions: 

1. Load the `leagues.csv` dataset and call it `leagues`
2. Create a `.copy()` of the `match` dataset and call it `match_teams`
3. Join the `match_teams` and the `leagues` dataset with a left join using their common keys
4. Find out the number of matches played by each league `name` in each `season`
5. Which combination of season-league has the lowest number of matches played?

In [ ]:
# 1.


In [ ]:
# 2.


In [ ]:
# 3.


In [ ]:
# 4.


In [ ]:
# 5.
